In [1]:
!pip install elasticsearch
!pip install faiss_cpu   # (thay vì faiss_gpu cho Mac M1 như mình đã giải thích)
!pip install nlp    # thay thế cho 'nlp'
!pip install transformers
!pip install apache_beam



In [6]:
import nlp
eli5 = nlp.load_dataset('eli5')
wiki40b_snippets = nlp.load_dataset('wiki_snippets', name='wiki40b_en_100_0')['train']

In [2]:
pip install git+https://github.com/huggingface/transformers.git


  Cloning https://github.com/huggingface/transformers.git to /private/var/folders/m4/1s7c462d1g77twlp95_rgycm0000gn/T/pip-req-build-m64alxp_
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /private/var/folders/m4/1s7c462d1g77twlp95_rgycm0000gn/T/pip-req-build-m64alxp_
  Resolved https://github.com/huggingface/transformers.git to commit fa3f2db5c7405a742fcb8f686d3754f70db00977
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.


In [3]:
import sys
sys.path.append('/Users/user')
from lfqa_utils import *


In [7]:
eli5['test_eli5'][12345]

{'q_id': '8houtx',
 'title': 'Why does water heated to room temperature feel colder than the air around it?',
 'selftext': '',
 'document': '',
 'subreddit': 'explainlikeimfive',
 'answers': {'a_id': ['dylcnfk', 'dylcj49'],
  'text': ["Water transfers heat more efficiently than air. When something feels cold it's because heat is being transferred from your skin to whatever you're touching. Since water absorbs the heat more readily than air, it feels colder.",
   "Air isn't as good at transferring heat compared to something like water or steel (sit on a room temperature steel bench vs. a room temperature wooden bench, and the steel one will feel more cold).\n\nWhen you feel cold, what you're feeling is heat being transferred out of you.  If there is no breeze, you feel a certain way.  If there's a breeze, you will get colder faster (because the moving air is pulling the heat away from you), and if you get into water, its quite good at pulling heat from you.   Get out of the water and ha

In [8]:
wiki40b_snippets[8991855]


{'_id': '{"nlp_id": 1665419, "wiki_id": "Q179635", "sp": 12, "sc": 653, "ep": 12, "ec": 1223}',
 'article_title': 'Heat transfer',
 'end_character': 1223,
 'end_paragraph': 12,
 'nlp_id': 1665419,
 'passage_text': 'from one place to another place without the movement of particles is called conduction, such as when placing a hand on a cold glass of water - heat is conducted from the warm skin to the cold glass, but if the hand is held a few inches from the glass, little conduction would occur since air is a poor conductor of heat. Steady state conduction is an idealized model of conduction that happens when the temperature difference driving the conduction is constant, so that after a time, the spatial distribution of temperatures in the conducting object does not change any',
 'section_title': 'Conduction',
 'start_character': 653,
 'start_paragraph': 12,
 'wiki_id': 'Q179635'}

In [9]:
es_client = Elasticsearch(
    [{'host': 'localhost', 'port': 9200, 'scheme': 'http'}],
)


In [10]:
from elasticsearch import Elasticsearch

# Kết nối đến Elasticsearch
es_client = Elasticsearch("http://localhost:9200")

# Đặt tên chỉ mục
index_name = 'wiki40b_snippets_100w'

# Cấu hình chỉ mục
index_config = {
    "settings": {
        "number_of_shards": 1,
    },
    "mappings": {
        "properties": {
            "article_title": {"type": "text"},
            "section_title": {"type": "text"},
            "passage_text": {"type": "text"},
        }
    }
}

# Tạo chỉ mục nếu chưa tồn tại
if not es_client.indices.exists(index=index_name):
    es_client.indices.create(index=index_name, body=index_config, headers={"Content-Type": "application/json"})

# Đặt truy vấn và số lượng kết quả
q = 'your query here'  # Đặt truy vấn của bạn vào đây
n_results = 10  # Số lượng kết quả bạn muốn

# Loại bỏ các từ bị cấm
banned = ['how', 'why', 'what', 'where', 'which', 'do', 'does', 'is', '?', 'eli5', 'eli5:']
q = ' '.join([w for w in q.split() if w not in banned])

# Gửi truy vấn tìm kiếm
response = es_client.search(
    index=index_name,
    body={
        "query": {
            "multi_match": {
                "query": q,
                "fields": ["article_title", "section_title", "passage_text^2"],
                "type": "cross_fields",
            }
        },
        "size": n_results,
    }
)

# Xử lý kết quả tìm kiếm
print(response)


{'took': 70, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}}


/var/folders/m4/1s7c462d1g77twlp95_rgycm0000gn/T/ipykernel_887/1593372765.py:36: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  response = es_client.search(


In [11]:
if not es_client.indices.exists(index='wiki40b_snippets_100w'):
    make_es_index_snippets(es_client, wiki40b_snippets, index_name='wiki40b_snippets_100w')



In [12]:
def query_es_index(question, es_client, index_name, size=10):
    query_body = {
        "size": size,  # Thêm tham số size vào đây
        "query": {
            "match": {
                "text": question
            }
        }
    }
    
    # Thực hiện truy vấn Elasticsearch
    res = es_client.search(index=index_name, body=query_body)
    
    # Xử lý kết quả trả về
    res_list = [
        {
            'article_title': hit['_source']['article_title'],
            'section_title': hit['_source']['section_title'],
            'passage_text': hit['_source']['passage_text']
        }
        for hit in res['hits']['hits']
    ]
    
    return res, res_list


In [13]:
question = eli5['test_eli5'][12345]['title']
doc, res_list = query_es_index(question, es_client, index_name='wiki40b_snippets_100w', size=10)


df = pd.DataFrame({
    'Article': ['---'] + [res['article_title'] for res in res_list],
    'Sections': ['---'] + [res['section_title'] if res['section_title'].strip() != '' else res['article_title']
                 for res in res_list],
    'Text': ['--- ' + question] + [res['passage_text'] for res in res_list],
})
df.style.set_properties(**{'text-align': 'left'})

/var/folders/m4/1s7c462d1g77twlp95_rgycm0000gn/T/ipykernel_887/1084859799.py:12: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res = es_client.search(index=index_name, body=query_body)


,Article,Sections,Text
0,---,---,--- Why does water heated to room temperature feel colder than the air around it?


In [14]:
import torch
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
print(f"PyTorch version: {torch.__version__}")
# Check PyTorch has access to MPS (Metal Performance Shader, Apple's GPU architecture)
print(f"Is MPS (Metal Performance Shader) built? {torch.backends.mps.is_built()}")
print(f"Is MPS available? {torch.backends.mps.is_available()}")
# Set the device      
device = "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using device: {device}")

PyTorch version: 2.1.2
Is MPS (Metal Performance Shader) built? True
Is MPS available? True
Using device: mps


In [13]:
class ArgumentsQAR():
    def __init__(self):
        self.batch_size = 16
        self.max_length = 128
        self.checkpoint_batch_size = 32
        self.print_freq = 100
        self.pretrained_model_name = "google/bert_uncased_L-8_H-768_A-12"
        self.model_save_name = "retriever_models/eli5_retriever_model_final"
        self.learning_rate = 2e-4
        self.num_epochs = 10

qar_args = ArgumentsQAR()

# Chuẩn bị Dataset
qar_train_dset = ELI5DatasetQARetriver(eli5['train_eli5'], training=True)
qar_valid_dset = ELI5DatasetQARetriver(eli5['validation_eli5'], training=False)

# Tải mô hình BERT đã pre-trained
qar_tokenizer, qar_model = make_qa_retriever_model(
    model_name=qar_args.pretrained_model_name,
    from_file=None,
    device="cpu"  # Sử dụng CPU
)

# Đảm bảo rằng model và tensor đều được chuyển sang CPU
device = torch.device("cpu")  # Xác định thiết bị là CPU
qar_model.to(device)  # Đảm bảo mô hình được chuyển sang CPU

# Hàm huấn luyện một epoch (chạy trên CPU)
def train_qa_retriever_epoch(model, dataset, tokenizer, optimizer, scheduler, args, epoch):
    device = torch.device("cpu")  # Chạy trên CPU
    model.train()
    
    for step, batch in enumerate(dataset):
        print(f"Step {step}, Batch: {batch}")  # In ra batch
        
        if len(batch) == 3:  # Giả sử batch chứa input_ids, attention_mask và labels
            input_ids, attention_mask, labels = batch
            
            # Chuyển các tensor sang CPU
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)
            
            # Huấn luyện
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            
            # Backpropagation
            loss.backward()
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
        else:
            print(f"Unexpected batch size: {len(batch)}")
            continue  # Bỏ qua nếu batch không đúng


# Hàm huấn luyện chính (chạy trên CPU)
def train_qa_retriever(qar_model, qar_tokenizer, qar_train_dset, qar_valid_dset, qar_args):
    optimizer = torch.optim.AdamW(qar_model.parameters(), lr=qar_args.learning_rate)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=100,
                                                num_training_steps=(qar_args.num_epochs + 1) * 
                                                math.ceil(len(qar_train_dset) / qar_args.batch_size))
    
    for epoch in range(qar_args.num_epochs):
        train_qa_retriever_epoch(qar_model, qar_train_dset, qar_tokenizer, optimizer, scheduler, qar_args, epoch)
    
    # Save only the final model (not optimizer or scheduler)
    print("Saving final model to {}".format(qar_args.model_save_name))
    torch.save(qar_model.state_dict(), qar_args.model_save_name)

# Bắt đầu huấn luyện trên CPU
train_qa_retriever(qar_model, qar_tokenizer, qar_train_dset, qar_valid_dset, qar_args)
